### 1. Loading Data

In [158]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# input image dimensions
img_rows, img_cols = 28, 28
num_classes=10

In [159]:
train_df = pd.read_csv('../data/mnist_train.csv', header=None)

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(
    train_df.iloc[:, 1:].values, train_df.iloc[:, 0].values,
    stratify=train_df.iloc[:, 0].values,
    train_size=0.8,
    random_state=42
)

X_raw_train.shape, X_raw_test.shape

((48000, 784), (12000, 784))

### 2.1. Keras Convolution Net
https://keras.io/examples/mnist_cnn/

In [160]:
import keras

In [161]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [162]:
y_train = keras.utils.to_categorical(y_raw_train, num_classes)
y_test = keras.utils.to_categorical(y_raw_test, num_classes)

X_train = X_raw_train.astype('float64') / 255
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)

X_test = X_raw_test.astype('float64') / 255
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((48000, 28, 28, 1), (12000, 28, 28, 1), (48000, 10), (12000, 10))

In [163]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [164]:
%%time
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 48s 1ms/step - loss: 0.3181 - accuracy: 0.9013 - val_loss: 0.0717 - val_accuracy: 0.9786
Epoch 2/10
48000/48000 [==============================] - 50s 1ms/step - loss: 0.0991 - accuracy: 0.9699 - val_loss: 0.0592 - val_accuracy: 0.9833
Epoch 3/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0739 - accuracy: 0.9771 - val_loss: 0.0525 - val_accuracy: 0.9849
Epoch 4/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0598 - accuracy: 0.9811 - val_loss: 0.0487 - val_accuracy: 0.9862
Epoch 5/10
48000/48000 [==============================] - 53s 1ms/step - loss: 0.0501 - accuracy: 0.9843 - val_loss: 0.0431 - val_accuracy: 0.9882
Epoch 6/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0441 - accuracy: 0.9861 - val_loss: 0.0431 - val_accuracy: 0.9886
Epoch 7/10
48000/48000 [==============================] - 53s 1ms/st

In [165]:
model.save('mnist.convnet.model')

### 2.2. Keras Dense

In [166]:
y_train = keras.utils.to_categorical(y_raw_train, num_classes)
y_test = keras.utils.to_categorical(y_raw_test, num_classes)

X_train = X_raw_train
X_test = X_raw_test

In [167]:
model = Sequential()
model.add(Dense(512, input_shape=(784,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [168]:
%%time
model.fit(X_raw_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(X_raw_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 3s 52us/step - loss: 6.6343 - accuracy: 0.8692 - val_loss: 1.0552 - val_accuracy: 0.9280
Epoch 2/20
48000/48000 [==============================] - 2s 48us/step - loss: 0.7077 - accuracy: 0.9253 - val_loss: 0.4627 - val_accuracy: 0.9467
Epoch 3/20
48000/48000 [==============================] - 2s 48us/step - loss: 0.3814 - accuracy: 0.9429 - val_loss: 0.4295 - val_accuracy: 0.9501
Epoch 4/20
48000/48000 [==============================] - 2s 49us/step - loss: 0.2692 - accuracy: 0.9544 - val_loss: 0.3910 - val_accuracy: 0.9557
Epoch 5/20
48000/48000 [==============================] - 3s 53us/step - loss: 0.2284 - accuracy: 0.9629 - val_loss: 0.3647 - val_accuracy: 0.9603
Epoch 6/20
48000/48000 [==============================] - 2s 48us/step - loss: 0.1745 - accuracy: 0.9690 - val_loss: 0.3754 - val_accuracy: 0.9607
Epoch 7/20
48000/48000 [==============================] - 2s 48us/st

In [169]:
model.save('mnist.dense512.model')

### 2.3. Keras Deep

In [170]:
y_train = keras.utils.to_categorical(y_raw_train, num_classes)
y_test = keras.utils.to_categorical(y_raw_test, num_classes)

X_train = X_raw_train
X_test = X_test

In [171]:
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [172]:
%%time
model.fit(X_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 6s 115us/step - loss: 0.4950 - accuracy: 0.8447 - val_loss: 0.2065 - val_accuracy: 0.9385
Epoch 2/12
48000/48000 [==============================] - 5s 113us/step - loss: 0.2181 - accuracy: 0.9335 - val_loss: 0.1558 - val_accuracy: 0.9532
Epoch 3/12
48000/48000 [==============================] - 6s 133us/step - loss: 0.1633 - accuracy: 0.9506 - val_loss: 0.1275 - val_accuracy: 0.9634
Epoch 4/12
48000/48000 [==============================] - 6s 129us/step - loss: 0.1347 - accuracy: 0.9591 - val_loss: 0.1258 - val_accuracy: 0.9643
Epoch 5/12
48000/48000 [==============================] - 5s 114us/step - loss: 0.1112 - accuracy: 0.9667 - val_loss: 0.1097 - val_accuracy: 0.9715
Epoch 6/12
48000/48000 [==============================] - 5s 115us/step - loss: 0.1025 - accuracy: 0.9683 - val_loss: 0.0999 - val_accuracy: 0.9705
Epoch 7/12
48000/48000 [==============================] - 6s 1

In [173]:
model.save('mnist.deep.dense.model')

### 3. Проверка сохраненной модели

#### Keras ConvNet Example

In [174]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

In [175]:
test_model = load_model('mnist.convnet.model')

test_df = pd.read_csv('../data/mnist_test.csv', header=None)
y_holdout = test_df.iloc[:, 0].values
X_holdout = test_df.iloc[:, 1:].values
X_holdout.shape, y_holdout.shape

((10000, 784), (10000,))

In [176]:
X_holdout = X_holdout.astype('float64') / 255
X_holdout = X_holdout.reshape(X_holdout.shape[0], 28, 28, 1)

y_pred = test_model.predict_classes(X_holdout)

In [177]:
accuracy_score(y_holdout, y_pred)

0.9915

#### One Dense

In [178]:
test_model = load_model('mnist.dense512.model')

test_df = pd.read_csv('../data/mnist_test.csv', header=None)
y_holdout = test_df.iloc[:, 0].values
X_holdout = test_df.iloc[:, 1:].values
X_holdout.shape, y_holdout.shape

((10000, 784), (10000,))

In [179]:
y_pred = test_model.predict_classes(X_holdout)

In [180]:
accuracy_score(y_holdout, y_pred)

0.9711

#### Deep Dense

In [181]:
test_model = load_model('mnist.deep.dense.model')

test_df = pd.read_csv('../data/mnist_test.csv', header=None)
y_holdout = test_df.iloc[:, 0].values
X_holdout = test_df.iloc[:, 1:].values
X_holdout.shape, y_holdout.shape

((10000, 784), (10000,))

In [182]:
y_pred = test_model.predict_classes(X_holdout)

In [183]:
accuracy_score(y_holdout, y_pred)

0.9801